# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4ca4f64850>
├── 'a' --> tensor([[-0.4081,  1.8504, -0.0261],
│                   [-0.2306,  0.8374, -0.5673]])
└── 'x' --> <FastTreeValue 0x7f4ca4f64e50>
    └── 'c' --> tensor([[ 0.4883, -1.6991, -1.1008,  1.6312],
                        [ 0.6688, -0.6292, -1.5267,  0.0973],
                        [-0.7176,  0.0279,  0.3726, -0.9494]])

In [4]:
t.a

tensor([[-0.4081,  1.8504, -0.0261],
        [-0.2306,  0.8374, -0.5673]])

In [5]:
%timeit t.a

64.4 ns ± 0.0644 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4ca4f64850>
├── 'a' --> tensor([[-0.6112,  1.0969,  0.4097],
│                   [ 0.2350, -0.1465, -0.8692]])
└── 'x' --> <FastTreeValue 0x7f4ca4f64e50>
    └── 'c' --> tensor([[ 0.4883, -1.6991, -1.1008,  1.6312],
                        [ 0.6688, -0.6292, -1.5267,  0.0973],
                        [-0.7176,  0.0279,  0.3726, -0.9494]])

In [7]:
%timeit t.a = new_value

73.7 ns ± 0.0166 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4081,  1.8504, -0.0261],
               [-0.2306,  0.8374, -0.5673]]),
    x: Batch(
           c: tensor([[ 0.4883, -1.6991, -1.1008,  1.6312],
                      [ 0.6688, -0.6292, -1.5267,  0.0973],
                      [-0.7176,  0.0279,  0.3726, -0.9494]]),
       ),
)

In [10]:
b.a

tensor([[-0.4081,  1.8504, -0.0261],
        [-0.2306,  0.8374, -0.5673]])

In [11]:
%timeit b.a

62.8 ns ± 0.0263 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2574, -0.7656, -0.9422],
               [-1.6217,  0.8526,  0.4803]]),
    x: Batch(
           c: tensor([[ 0.4883, -1.6991, -1.1008,  1.6312],
                      [ 0.6688, -0.6292, -1.5267,  0.0973],
                      [-0.7176,  0.0279,  0.3726, -0.9494]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.07 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.1 µs ± 4.74 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 876 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 370 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4ca53527f0>
├── 'a' --> tensor([[[-0.4081,  1.8504, -0.0261],
│                    [-0.2306,  0.8374, -0.5673]],
│           
│                   [[-0.4081,  1.8504, -0.0261],
│                    [-0.2306,  0.8374, -0.5673]],
│           
│                   [[-0.4081,  1.8504, -0.0261],
│                    [-0.2306,  0.8374, -0.5673]],
│           
│                   [[-0.4081,  1.8504, -0.0261],
│                    [-0.2306,  0.8374, -0.5673]],
│           
│                   [[-0.4081,  1.8504, -0.0261],
│                    [-0.2306,  0.8374, -0.5673]],
│           
│                   [[-0.4081,  1.8504, -0.0261],
│                    [-0.2306,  0.8374, -0.5673]],
│           
│                   [[-0.4081,  1.8504, -0.0261],
│                    [-0.2306,  0.8374, -0.5673]],
│           
│                   [[-0.4081,  1.8504, -0.0261],
│                    [-0.2306,  0.8374, -0.5673]]])
└── 'x' --> <FastTreeValue 0x7f4bd64d4f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.5 µs ± 72.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4bdd1d8c70>
├── 'a' --> tensor([[-0.4081,  1.8504, -0.0261],
│                   [-0.2306,  0.8374, -0.5673],
│                   [-0.4081,  1.8504, -0.0261],
│                   [-0.2306,  0.8374, -0.5673],
│                   [-0.4081,  1.8504, -0.0261],
│                   [-0.2306,  0.8374, -0.5673],
│                   [-0.4081,  1.8504, -0.0261],
│                   [-0.2306,  0.8374, -0.5673],
│                   [-0.4081,  1.8504, -0.0261],
│                   [-0.2306,  0.8374, -0.5673],
│                   [-0.4081,  1.8504, -0.0261],
│                   [-0.2306,  0.8374, -0.5673],
│                   [-0.4081,  1.8504, -0.0261],
│                   [-0.2306,  0.8374, -0.5673],
│                   [-0.4081,  1.8504, -0.0261],
│                   [-0.2306,  0.8374, -0.5673]])
└── 'x' --> <FastTreeValue 0x7f4be1fdeaf0>
    └── 'c' --> tensor([[ 0.4883, -1.6991, -1.1008,  1.6312],
                        [ 0.6688, -0.6292, -1.5267,  0.0973],
                 

In [23]:
%timeit t_cat(trees)

31.4 µs ± 67.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.4 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4081,  1.8504, -0.0261],
                [-0.2306,  0.8374, -0.5673]],
       
               [[-0.4081,  1.8504, -0.0261],
                [-0.2306,  0.8374, -0.5673]],
       
               [[-0.4081,  1.8504, -0.0261],
                [-0.2306,  0.8374, -0.5673]],
       
               [[-0.4081,  1.8504, -0.0261],
                [-0.2306,  0.8374, -0.5673]],
       
               [[-0.4081,  1.8504, -0.0261],
                [-0.2306,  0.8374, -0.5673]],
       
               [[-0.4081,  1.8504, -0.0261],
                [-0.2306,  0.8374, -0.5673]],
       
               [[-0.4081,  1.8504, -0.0261],
                [-0.2306,  0.8374, -0.5673]],
       
               [[-0.4081,  1.8504, -0.0261],
                [-0.2306,  0.8374, -0.5673]]]),
    x: Batch(
           c: tensor([[[ 0.4883, -1.6991, -1.1008,  1.6312],
                       [ 0.6688, -0.6292, -1.5267,  0.0973],
                       [-0.7176,  0.0279,  0.3726, -0.9494]],
         

In [26]:
%timeit Batch.stack(batches)

78.9 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4081,  1.8504, -0.0261],
               [-0.2306,  0.8374, -0.5673],
               [-0.4081,  1.8504, -0.0261],
               [-0.2306,  0.8374, -0.5673],
               [-0.4081,  1.8504, -0.0261],
               [-0.2306,  0.8374, -0.5673],
               [-0.4081,  1.8504, -0.0261],
               [-0.2306,  0.8374, -0.5673],
               [-0.4081,  1.8504, -0.0261],
               [-0.2306,  0.8374, -0.5673],
               [-0.4081,  1.8504, -0.0261],
               [-0.2306,  0.8374, -0.5673],
               [-0.4081,  1.8504, -0.0261],
               [-0.2306,  0.8374, -0.5673],
               [-0.4081,  1.8504, -0.0261],
               [-0.2306,  0.8374, -0.5673]]),
    x: Batch(
           c: tensor([[ 0.4883, -1.6991, -1.1008,  1.6312],
                      [ 0.6688, -0.6292, -1.5267,  0.0973],
                      [-0.7176,  0.0279,  0.3726, -0.9494],
                      [ 0.4883, -1.6991, -1.1008,  1.6312],
                      [ 0.6688, -

In [28]:
%timeit Batch.cat(batches)

151 µs ± 290 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 897 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
